<table align="left"><td>
  <a target="_blank"  href="https://github.com/Dhruv0208/Research-Project/Autoencoder.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on github
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/Dhruv0208/Research-Project/Autoencoder.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
</td></table>

In [ ]:
import sys
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install imutils
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install -q tfds-nightly tensorflow
!{sys.executable} -m pip install pydot
!{sys.executable} -m pip install graphviz
!{sys.executable} -m pip install pydotplus

In [1]:
import keras
from tensorflow.keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.layers import Input
from keras.utils.vis_utils import plot_model
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import json
import os
from imutils import paths
import cv2
import IPython.display as display
import multiprocessing

In [2]:
print(tf.__version__)

2.3.1


In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}',format(device_name))

Found GPU at: {} /device:GPU:0


In [ ]:
splits = ['train', 'validation']
(train_dataset, val_dataset), dataset_info = tfds.load(name='coco/2017', data_dir = '../dhruv', 
                                                       split = list(splits), 
                                                       download=False, with_info=True,
                                                                    as_supervised=False)
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(val_dataset, tf.data.Dataset)
print(train_dataset)
print(val_dataset)

In [4]:
file_pattern = f'coco/2017/1.1.0/coco-train.tfrecord*'

In [5]:
files = tf.data.Dataset.list_files(file_pattern)

In [ ]:
len(files)

In [6]:
train_ds = files.interleave(tf.data.TFRecordDataset,
                           cycle_length = 4,
                           num_parallel_calls = tf.data.experimental.AUTOTUNE)

In [7]:
train_ds

<ParallelInterleaveDataset shapes: (), types: tf.string>

In [9]:
def read_tfrecord(serialized_example):
    feature_description={
        "image": tf.io.FixedLenFeature((), tf.string, "")
    }
    example = tf.io.parse_single_example(
    serialized_example, feature_description
    )
    image = tf.io.decode_jpeg(example['image'], channels=3)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = image / 255
    image = tf.image.resize(image, (512, 512))
    return image

In [10]:
cores = multiprocessing.cpu_count()
print(cores)

train_ds = train_ds.map(read_tfrecord, num_parallel_calls=cores)
train_ds = train_ds.cache()

12


In [ ]:
for i in train_ds:
    print(i)

In [11]:
train_ds = train_ds.shuffle(1024).batch(32)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [12]:
train_ds

<PrefetchDataset shapes: (None, 512, 512, 3), types: tf.float32>

## Preparing autoencoder

In [13]:
input_img = Input(shape=(512,512,3))
x = layers.Conv2D(256, (1,1), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2,2), padding = 'same')(x)
x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2,2), padding = 'same')(x)
x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2,2), padding = 'same')(x)
x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2,2), padding = 'same')(x)
x = layers.Conv2D(16, (3,3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2,2), padding='same')(x)
x = layers.Conv2D(16, (3,3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2,2), padding='same')(x)
x = layers.Conv2D(8, (3,3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2,2), padding='same')(x)

##################################################################

x = layers.Conv2D(8, (3,3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2,2))(x)
x = layers.Conv2D(16, (3,3), activation='relu', padding = 'same')(x)
x = layers.UpSampling2D((2,2))(x)
x = layers.Conv2D(16, (3,3), activation = 'relu', padding = 'same')(x)
x = layers.UpSampling2D((2,2))(x)
x = layers.Conv2D(32, (3,3), activation = 'relu', padding= 'same')(x)
x = layers.UpSampling2D((2,2))(x)
x = layers.Conv2D(64, (3,3), activation = 'relu', padding= 'same')(x)
x = layers.UpSampling2D((2,2))(x)
x = layers.Conv2D(128, (3,3), activation = 'relu', padding= 'same')(x)
x = layers.UpSampling2D((2,2))(x)
x = layers.Conv2D(256, (3,3), activation = 'relu', padding= 'same')(x)
x = layers.UpSampling2D((2,2))(x)
decoded = layers.Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)

autoencoder = tf.keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

In [ ]:
autoencoder.summary()

In [ ]:
plot_model(autoencoder, show_shapes=True)

## Extracting Features from intermediate layers

In [ ]:
intermediate_layer_model_1 = keras.Model(inputs=autoencoder.inputs,
                                      outputs=autoencoder.get_layer('max_pooling2d_1').output)
features_1 = intermediate_layer_model_1(train_ds)
intermediate_layer_model_2 = keras.Model(inputs=autoencoder.inputs,
                                      outputs=autoencoder.get_layer('max_pooling2d_3').output)
features_2 = intermediate_layer_model_1(train_ds)
intermediate_layer_model_3 = keras.Model(inputs=autoencoder.inputs,
                                      outputs=autoencoder.get_layer('max_pooling2d_5').output)
features_3 = intermediate_layer_model_1(train_ds)

## Training and testing the model

In [ ]:
print(tf.compat.v1.trainable_variables())

In [14]:
for i in train_ds:
    autoencoder.fit(x=i, epochs = 2, shuffle = True, verbose=1)

Epoch 1/2


ValueError: in user code:

    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:789 run_step  **
        outputs = model.t

## Comparing test and model created images

In [ ]:
n = 2
plt.figure(figsize=(32,32))
for i in range(n):
    # display original
    ax = plt.subplot(3, n, i+1)
    plt.imshow(test_images[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    
    # display reconstruction
    ax = plt.subplot(3, n, i+n+1)
    plt.imshow(decoded_imgs[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
plt.show()

## Visualizing loss

In [ ]:
print(history.history.keys())

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()